## Calculate GAM in metabolic model
    **Version:** 1.0
    **Author:** Shakra Ahmad
    ** Modified by:** Sol Han
    **Organization:** Sybirg Lab
    **Description:** This code is used to calculate GAM in metabolic model.

    Copyright (c) 2023 Sybirg Lab, Konkuk University 
    All rights reserved. 

    **Note: This code is provided as-is, without any warranties or guarantees.*

In [55]:
# metabolic model open
import cobra

s12 = cobra.io.read_sbml_model('C:\\Users\\twall\\Downloads\\iSH1474_GAM_NGAM.xml')
model = s12.copy()

In [58]:
# Change the coefficients of amino acids and dnas in biomass equations

aa = ['ala__L_c','arg__L_c','asn__L_c','asp__L_c','cys__L_c','gln__L_c','glu__L_c','gly_c','his__L_c','ile__L_c',
       'leu__L_c','lys__L_c','met__L_c','phe__L_c','pro__L_c','ser__L_c','thr__L_c','trp__L_c','tyr__L_c','val__L_c']
dna = ['datp_c','dctp_c','dgtp_c','dttp_c']

aa_coeffs = {'ala__L_c':-0.609649,'arg__L_c':-0.363607,'asn__L_c':-0.160967,'asp__L_c':-0.291447,'cys__L_c':-0.058177,
          'gln__L_c':-0.257130,'glu__L_c':-0.310237,'gly_c':-0.436021,'his__L_c':-0.129586,'ile__L_c':-0.249629,
       'leu__L_c':-0.644223,'lys__L_c':-0.185472,'met__L_c':-0.127920,'phe__L_c':-0.194990,'pro__L_c':-0.267482,
          'ser__L_c':-0.310657,'thr__L_c':-0.259012,'trp__L_c':-0.079935,'tyr__L_c':-0.139980,'val__L_c':-0.393654}

dna_coeffs = {'datp_c':-0.010765,'dctp_c':-0.017418,'dgtp_c':-0.017382,'dttp_c':-0.010745}

biomass_equations = ['BIOMASS_KT2440_Core2','BIOMASS_KT2440_WT3']

for eq in biomass_equations:
    
    delete = {}

    for m in aa+dna:
        globals()[m] = model.metabolites.get_by_id(m)
        globals()[m+'_coef'] = model.reactions.get_by_id(eq).get_coefficient(globals()[m])
        delete[m]=-(globals()[m+'_coef'])
    
    model.reactions.get_by_id(eq).add_metabolites(delete)
    model.reactions.get_by_id(eq).add_metabolites(aa_coeffs)
    model.reactions.get_by_id(eq).add_metabolites(dna_coeffs)
    
    print(eq, model.reactions.get_by_id(eq).reaction)

0.000223 10fthf_c + 0.000223 2fe2s_c + 0.000223 4fe4s_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.609649 ala__L_c + 0.000223 amet_c + 0.363607 arg__L_c + 0.160967 asn__L_c + 0.291447 asp__L_c + 46.478683 atp_c + 0.000223 btamp_c + 0.004226 ca2_c + 0.000223 chor_c + 0.004226 cl_c + 0.000167 coa_c + 0.002817 cobalt2_c + 0.159783 ctp_c + 0.002817 cu2_c + 0.058177 cys__L_c + 0.010765 datp_c + 0.017418 dctp_c + 0.017382 dgtp_c + 0.010745 dttp_c + 0.000223 fad_c + 0.00634 fe2_c + 0.00634 fe3_c + 0.25713 gln__L_c + 0.310237 glu__L_c + 0.436021 gly_c + 0.210704 gtp_c + 40.8402 h2o_c + 0.000223 hemeO_c + 0.129586 his__L_c + 0.249629 ile__L_c + 0.158509 k_c + 0.644223 leu__L_c + 3e-06 lipopb_c + 0.185472 lys__L_c + 3.1e-05 malcoa_c + 0.12792 met__L_c + 0.007044 mg2_c + 0.000223 mlthf_c + 0.002817 mobd_c + 0.012699 murein5px4p_p + 0.003522 na1_c + 0.001786 nad_c + 4.4e-05 nadh_c + 0.000111 nadp_c + 0.000335 nadph_c + 0.010567 nh4_c + 0.002817 ni2_c + 0.00858 pe160_c + 0.042623 pe161_c + 0.002855 

In [38]:
# q= growth_rate/Y_G + maintenance coefficient(mc)

inverse_Y_G = 10
mc = 0.077



# NGAM calculation

import numpy as np

biomass_equations = ['BIOMASS_KT2440_Core2','BIOMASS_KT2440_WT3']

for eq in biomass_equations:
    for m in gam_am:
        globals()[m] = model.metabolites.get_by_id(m)
        globals()[m+'_coef'] = model.reactions.get_by_id(eq).get_coefficient(globals()[m])

    with model:
        model.reactions.get_by_id('ATPM').bounds = (-99999,99999)
        model.reactions.get_by_id(eq).add_metabolites({'h2o_c':-h2o_c_coef,'atp_c':-atp_c_coef,'adp_c':-adp_c_coef,'h_c':-h_c_coef,'pi_c':-pi_c_coef})
    
        model.objective = 'ATPM'
        model.reactions.get_by_id("EX_glc__D_e").lower_bound = -mc
        model.reactions.get_by_id('BIOMASS_KT2440_Core2').lower_bound = 0
        model.reactions.get_by_id('BIOMASS_KT2440_WT3').lower_bound = 0
        ngam = round(model.optimize().objective_value,2)
        
print('NGAM:',ngam)

C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\U

C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\U

C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\U

C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\U

C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\U

...

C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\U

C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\U

C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\U

C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\twall\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\U

NGAM: 1.67
GAM: 49.98999999999801


In [74]:
# GAM calculation

glucose = [1,2,3,4,5] #mmol

met = ['h2o_c','atp_c','adp_c','h_c','pi_c'] # gam associated metabolites
biomass = model.reactions.get_by_id('BIOMASS_KT2440_Core2')
model.objective = 'BIOMASS_KT2440_Core2'

for m in met:
    globals()[m] = model.metabolites.get_by_id(m)
    globals()[m+'_coef'] = biomass.get_coefficient(globals()[m])

for coef in (np.arange(40,50,0.01)):
    objective_values = []
    
    with model:
        model.reactions.get_by_id('ATPM').bounds = (ngam,ngam)
        for glc in glucose:
            model.reactions.get_by_id("EX_glc__D_e").lower_bound = -glc
            biomass.add_metabolites({'h2o_c':-(coef-5.4698)-h2o_c_coef,'atp_c':-(coef+0.168683)-atp_c_coef,'adp_c':coef-adp_c_coef,'h_c':coef-h_c_coef,'pi_c':coef-pi_c_coef-0.003522})
            objective_values.append(model.optimize().objective_value)
            
        slope, intercept = np.polyfit(objective_values, glucose,1)
        
        print('gam:',coef, 'slope:',slope)

gam: 40.0 slope: 8.612560436561704
gam: 40.01 slope: 8.615186468617233
gam: 40.019999999999996 slope: 8.617812655651127
gam: 40.029999999999994 slope: 8.620438997660784
gam: 40.03999999999999 slope: 8.62306549464365
gam: 40.04999999999999 slope: 8.625692146597187
gam: 40.05999999999999 slope: 8.628318953518894
gam: 40.069999999999986 slope: 8.63094591540618
gam: 40.079999999999984 slope: 8.633573032256558
gam: 40.08999999999998 slope: 8.636200304067346
gam: 40.09999999999998 slope: 8.638827730836088
gam: 40.10999999999998 slope: 8.64145531256056
gam: 40.119999999999976 slope: 8.644083049237908
gam: 40.129999999999974 slope: 8.646710940865542
gam: 40.13999999999997 slope: 8.649338987441295
gam: 40.14999999999997 slope: 8.651967188962164
gam: 40.15999999999997 slope: 8.654595545426202
gam: 40.169999999999966 slope: 8.657224056830497
gam: 40.179999999999964 slope: 8.65985272317254
gam: 40.18999999999996 slope: 8.662481544450047
gam: 40.19999999999996 slope: 8.665110520660289
gam: 40.20999

gam: 41.71999999999966 slope: 9.066515017738864
gam: 41.729999999999656 slope: 9.069167670450545
gam: 41.739999999999654 slope: 9.071820477734185
gam: 41.74999999999965 slope: 9.074473439587603
gam: 41.75999999999965 slope: 9.077126556008777
gam: 41.76999999999965 slope: 9.079779826995294
gam: 41.779999999999646 slope: 9.082433252544838
gam: 41.789999999999644 slope: 9.085086832655573
gam: 41.79999999999964 slope: 9.087740567325133
gam: 41.80999999999964 slope: 9.090394456551419
gam: 41.81999999999964 slope: 9.09304850033205
gam: 41.829999999999636 slope: 9.095702698665006
gam: 41.839999999999634 slope: 9.098357051548177
gam: 41.84999999999963 slope: 9.101011558979177
gam: 41.85999999999963 slope: 9.103666220956134
gam: 41.86999999999963 slope: 9.106321037476624
gam: 41.879999999999626 slope: 9.108976008538624
gam: 41.889999999999624 slope: 9.11163113413975
gam: 41.89999999999962 slope: 9.114286414278157
gam: 41.90999999999962 slope: 9.116941848951251
gam: 41.91999999999962 slope: 9.11

gam: 43.42999999999932 slope: 9.522363599455591
gam: 43.439999999999316 slope: 9.525042653507702
gam: 43.449999999999314 slope: 9.527721861781307
gam: 43.45999999999931 slope: 9.5304012242743
gam: 43.46999999999931 slope: 9.533080740985069
gam: 43.47999999999931 slope: 9.535760411911216
gam: 43.489999999999306 slope: 9.538440237051296
gam: 43.499999999999304 slope: 9.541120216403206
gam: 43.5099999999993 slope: 9.543800349965045
gam: 43.5199999999993 slope: 9.546480637734735
gam: 43.5299999999993 slope: 9.549161079710778
gam: 43.539999999999296 slope: 9.551841675890925
gam: 43.549999999999294 slope: 9.55452242627329
gam: 43.55999999999929 slope: 9.557203330856172
gam: 43.56999999999929 slope: 9.559884389637466
gam: 43.57999999999929 slope: 9.562565602615301
gam: 43.589999999999286 slope: 9.565246969788118
gam: 43.599999999999284 slope: 9.567928491153523
gam: 43.60999999999928 slope: 9.57061016670986
gam: 43.61999999999928 slope: 9.573291996455197
gam: 43.62999999999928 slope: 9.5759739

gam: 45.13999999999898 slope: 9.982721927216458
gam: 45.149999999998975 slope: 9.985427326357822
gam: 45.15999999999897 slope: 9.988132879412081
gam: 45.16999999999897 slope: 9.99083858637732
gam: 45.17999999999897 slope: 9.99354444725169
gam: 45.18999999999897 slope: 9.996250462033546
gam: 45.199999999998965 slope: 9.998956630721192
gam: 45.20999999999896 slope: 10.001662953313312
gam: 45.21999999999896 slope: 10.00436942980773
gam: 45.22999999999896 slope: 10.007076060202852
gam: 45.23999999999896 slope: 10.009782844497268
gam: 45.249999999998956 slope: 10.012489782688808
gam: 45.25999999999895 slope: 10.01519687477621
gam: 45.26999999999895 slope: 10.017904120757406
gam: 45.27999999999895 slope: 10.020611520631137
gam: 45.28999999999895 slope: 10.023319074395308
gam: 45.299999999998946 slope: 10.026026782048712
gam: 45.309999999998944 slope: 10.028734643589331
gam: 45.31999999999894 slope: 10.03144265901528
gam: 45.32999999999894 slope: 10.03415082832515
gam: 45.33999999999894 slope

gam: 46.81999999999864 slope: 10.439386783907091
gam: 46.82999999999864 slope: 10.442118017340732
gam: 46.83999999999864 slope: 10.444849404415981
gam: 46.84999999999864 slope: 10.447580945130985
gam: 46.859999999998635 slope: 10.450312639484837
gam: 46.86999999999863 slope: 10.453044487475376
gam: 46.87999999999863 slope: 10.45577648910148
gam: 46.88999999999863 slope: 10.458508644361427
gam: 46.89999999999863 slope: 10.461240953253714
gam: 46.909999999998625 slope: 10.463973415777149
gam: 46.91999999999862 slope: 10.466706031929768
gam: 46.92999999999862 slope: 10.469438801710186
gam: 46.93999999999862 slope: 10.47217172511688
gam: 46.94999999999862 slope: 10.474904802148352
gam: 46.959999999998615 slope: 10.477638032803116
gam: 46.96999999999861 slope: 10.480371417079382
gam: 46.97999999999861 slope: 10.483104954975975
gam: 46.98999999999861 slope: 10.485838646491247
gam: 46.99999999999861 slope: 10.488572491624003
gam: 47.009999999998605 slope: 10.491306490371919
gam: 47.0199999999

gam: 48.48999999999831 slope: 10.897631252302114
gam: 48.49999999999831 slope: 10.90038812329652
gam: 48.50999999999831 slope: 10.903145147687974
gam: 48.519999999998305 slope: 10.90590232547573
gam: 48.5299999999983 slope: 10.908659656658038
gam: 48.5399999999983 slope: 10.911417141233269
gam: 48.5499999999983 slope: 10.914174779200547
gam: 48.5599999999983 slope: 10.916932570558219
gam: 48.569999999998295 slope: 10.919690515305012
gam: 48.57999999999829 slope: 10.922448613439235
gam: 48.58999999999829 slope: 10.925206864959756
gam: 48.59999999999829 slope: 10.927965269864954
gam: 48.60999999999829 slope: 10.930723828153962
gam: 48.619999999998285 slope: 10.933482539824457
gam: 48.62999999999828 slope: 10.936241404876073
gam: 48.63999999999828 slope: 10.939000423306775
gam: 48.64999999999828 slope: 10.941759595115604
gam: 48.65999999999828 slope: 10.944518920300649
gam: 48.669999999998275 slope: 10.94727839886114
gam: 48.67999999999827 slope: 10.950038030795161
gam: 48.68999999999827 

In [76]:
#fix the GAM/NGAM

gam = 45.21

for eq in biomass_equations:
    for m in met:
        globals()[m] = model.metabolites.get_by_id(m)
        globals()[m+'_coef'] = model.reactions.get_by_id(eq).get_coefficient(globals()[m])
    model.reactions.get_by_id(eq).add_metabolites({'h2o_c':-h2o_c_coef,'atp_c':-atp_c_coef,'adp_c':-adp_c_coef,'h_c':-h_c_coef,'pi_c':-pi_c_coef})

    model.reactions.get_by_id(eq).add_metabolites({'h2o_c':round(-(gam-5.4698),6),
                                                                 'atp_c':round(-(gam+0.168683),6),
                                                                 'adp_c':round(gam,6),
                                                                 'h_c':round(gam,6),
                                                                 'pi_c':round(gam-0.003522,6)})
    print(model.reactions.get_by_id(eq).get_coefficient(adp_c))
    
model.reactions.get_by_id('ATPM').bounds = (ngam, 999999.0)


# save the model

model.objective = 'BIOMASS_KT2440_WT3'

cobra.io.write_sbml_model(model, 'iSH1474_GAM_NGAM.xml')
cobra.io.save_matlab_model(model, 'iSH1474_GAM_NGAM.mat')
cobra.io.save_json_model(model, 'iSH1474_GAM_NGAM.json')

45.21
45.21
